In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import os

In [92]:
root_dir = r'/kaggle/input/stanford-dogs-dataset/images/Images'
trans = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((224,224))]
)

batch_size = 4

data= torchvision.datasets.ImageFolder(root= root_dir, transform= trans)
train_data,test_data= torch.utils.data.random_split(data, [17580,3000])
traindataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True,)
testdataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          shuffle=True,)



# # Get a list of all files in the directory
classes = os.listdir(root_dir)
classes=[x.split('-')[-1] for x in classes]

In [29]:
len(data)

20580

In [33]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 5, 5,padding='same')
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(5, 12, 5,padding='same')
        self.conv3 = nn.Conv2d(12, 3, 5,padding='same')
        self.conv4 = nn.Conv2d(3, 234, 5,padding='same')
        self.fc1 = nn.Linear(183456, 201)
        self.fc2 = nn.Linear(201, 150)
        self.fc3 = nn.Linear(150, 120)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool1(F.relu(self.conv3(x)))
        x = self.pool1(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x =self.fc3(x)
        return x


net = Net()

In [34]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

In [35]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 0.479
[1,   400] loss: 0.479
[1,   600] loss: 0.479
[1,   800] loss: 0.479
[1,  1000] loss: 0.479
[2,   200] loss: 0.479
[2,   400] loss: 0.479
[2,   600] loss: 0.479
[2,   800] loss: 0.479
[2,  1000] loss: 0.479
Finished Training


In [91]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testdataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        outputs=[(o.argmax()) for o in outputs]
        total += len(labels)
        for x in range( len(outputs)):
            if outputs[x]== labels[x]:
                correct+=1
        

print(f'Accuracy of the network on test images: {(100 * correct )/ total} %')

Accuracy of the network on test images: 0.9666666666666667 %
